In [1]:
import pandas as pd
import folium
import requests
import numpy as np
import folium.plugins

batting = pd.read_csv('Batting.csv')
player_info_df = pd.read_csv('People.csv')
batting_career = []
listofplayers = []
list_of_names = []
players_already_entered = []
i = 0
i=i
list_of_positions = ['3b','2b','1b','ss','c','lf','cf','rf']
appearances = pd.read_csv("Appearances.csv")
apperances_career = pd.read_csv("career_appearances")
player_id = player_info_df['playerID']
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
batting_career = pd.read_csv('batting_career')


In [2]:
def player_sort_by_stat(player_list, desired_players, stat, number_of_players,batting_career):
    i = 0
    best_for_position = []
    for player in player_list.to_records():
        if player['playerID'] in desired_players and i < number_of_players:
            player_stats = {'playerID' : player['playerID'], stat : player[stat]}
            best_for_position.append(player_stats)
            i += 1
        elif i >= number_of_players:
            break
    return best_for_position

In [3]:
def player_search(position):
    desired_players = []
    position = position.lower()
    position_key = "G_" + position
    for player in apperances_career.to_records():
        player_position = max_position(player)
        if position_key == player_position:
            name = player['playerID']
            desired_players.append(name)
    return desired_players 

In [4]:
def max_position(player):
    max_position = -1
    if int(player['G_p']) > max_position:
        max_position = int(player['G_p'])
        position = "G_p"
    if int(player['G_c']) > max_position:
        max_position = int(player['G_c'])
        position = "G_c"
    if int(player['G_1b']) > max_position:
        max_position = int(player['G_1b'])
        position = "G_1b"
    if int(player['G_2b']) > max_position:
        max_position = int(player['G_2b'])
        position = "G_2b"
    if int(player['G_ss']) > max_position:
        max_position = int(player['G_ss'])
        position = "G_ss"
    if int(player['G_3b']) > max_position:
        max_position = int(player['G_3b'])
        position = "G_3b"
    if int(player['G_lf']) > max_position:
        max_position = int(player['G_lf'])
        position = "G_lf"
    if int(player['G_cf']) > max_position:
        max_position = int(player['G_cf'])
        position = "G_cf"
    if int(player['G_rf']) > max_position:
        max_position = int(player['G_rf'])
        position = "G_rf"
    return position 

In [5]:
def geocode(birthCity, birthState):
    text = ("%s , %s"%(birthCity, birthState))
    data =  { 'q' : text, 'format' : 'json'}
    url='https://nominatim.openstreetmap.org/search' 
    response = requests.get(url, params=data)
    geodata = response.json()
    lat = float(geodata[0]['lat'])
    lon = float(geodata[0]['lon'])
    lat_lon = {'lat': lat , 'lon': lon}
    return lat_lon

In [6]:
def player_map(listofplayers):
    map = folium.Map(location=(39.8283, -98.5795), zoom_start=4.21)
    for player in player_info_df.to_records():
        if player['playerID'] in listofplayers:
            birthCity = player['birthCity']
            birthState = player['birthState']
            if player['birthCountry'] == "USA":
                birthState = states[birthState]
            try:
                first_name = player['nameFirst']
                last_name = player['nameLast']
                name = first_name + " " + last_name
                player_lat_lon = geocode(birthCity, birthState)
                pos = (player_lat_lon['lat'],player_lat_lon['lon']) 
                message = "%s, %s-%s %s, %s " % (name,player['birthYear'], player['deathYear'],birthCity, birthState)
                marker = folium.Marker(location=pos, 
                        popup=message
                              )
                map.add_children(marker)
            except:
                pass
    return map

In [7]:
print("---MLB Stat Leaders by Position---")
while True:
    print(list_of_positions)
    position = input("Please input a positon see above or 'quit':  ")
    position = position.lower()
    if position == 'quit':
        print("Quitting...")
        break
    else:
        if position in list_of_positions:
            desired_players = player_search(position)
        else:
            print("Please Input a Valid Postition")
            break
    print(batting_career.keys())
    stat = input("Please Input the Stat You Would Like to See 'H, HR, AB':")
    stat = stat.upper()
    if stat == 'QUIT':
        print("Quitting...")
        break
    try:
        player_list = batting_career.sort_values(by=[stat], ascending=False)
    except:
        print("Please Input a Valid Stat")
        break
    number_of_players = input("How Many Players Would You Like To See? ")
    if number_of_players.lower() == 'quit':
        print("Quitting...")
        break
    try:
        number_of_players = int(number_of_players)
        best_of_players = player_sort_by_stat(player_list, desired_players, stat, number_of_players, batting_career)
    except:
        print("Please Input an Integer")
        break
    best_of_players_df = pd.DataFrame(best_of_players)
    for player_id in best_of_players_df['playerID']:
        listofplayers.append(player_id)
        for player in player_info_df.to_records():
            if player['playerID'] == player_id: 
                name = player['nameFirst'] + ' ' + player['nameLast']
                list_of_names.append(name)
                break
    best_of_players_df['Name'] = list_of_names
    list_of_names = []
    print(best_of_players_df)
    break    
player_map(listofplayers)

---MLB Stat Leaders by Position---
['3b', '2b', '1b', 'ss', 'c', 'lf', 'cf', 'rf']
Please input a positon see above or 'quit':  ss
Index(['AB', 'AVG', 'G', 'H', 'HR', 'R', 'RBI', 'SB', 'playerID'], dtype='object')
Please Input the Stat You Would Like to See 'H, HR, AB':h
How Many Players Would You Like To See? 5
      H   playerID            Name
0  3465  jeterde01     Derek Jeter
1  3184  ripkeca01      Cal Ripken
2  3142  yountro01     Robin Yount
3  3115  rodrial01  Alex Rodriguez
4  2877  vizquom01    Omar Vizquel


c:\users\bency\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.


In [29]:
i


0